# plotting quantified data

In [ ]:
import xarray as xr
import pint_xarray
from pint_xarray import unit_registry as ureg

xr.set_options(display_expand_data=False)

## load the data

In [ ]:
ds = xr.tutorial.open_dataset("air_temperature")
data = ds.air
data

## convert units into a format understood by pint

<div class="alert alert-info">
<strong>Note:</strong> this example uses the data provided by the <code>xarray.tutorial</code> functions. As such, the <code>units</code> attributes follow the CF conventions, which <code>pint</code> does not understand by default. To work around that, we are modifying the <code>units</code> attributes here, but in general it is better to use a library that adds support for the units used by the CF conventions to <code>pint</code>.
</div>

In [ ]:
data.lat.attrs["units"] = "degree"
data.lon.attrs["units"] = "degree"

## quantify the data

In [ ]:
quantified = data.pint.quantify()
quantified

## work with the data

In [ ]:
monthly_means = (
    quantified
    .pint.to("degC")
    .sel(time="2013")
    .groupby("time.month").mean()
)
monthly_means

Most operations will preserve the units but there are some which will drop them (see the [duck array integration status](https://xarray.pydata.org/en/stable/user-guide/duckarrays.html#missing-features) page). To work around that there are unit-aware versions on the `.pint` accessor. For example, to select data use `.pint.sel` instead of `.sel`:

In [ ]:
monthly_means.pint.sel(
    lat=ureg.Quantity(4350, "angular_minute"),
    lon=ureg.Quantity(12000, "angular_minute"),
)

## plot

`xarray`'s plotting functions will cast the data to `numpy.ndarray`, so we need to "dequantify" first.

In [ ]:
monthly_means.pint.dequantify(format="~P").plot.imshow(col="month", col_wrap=4)